# Prova – Projetista de Sistemas Embarcados ML (Mobit)

- Q1: OpenCV (contagem por contornos)
- Q2.1: YOLOv8 (contagem de pessoas)
- Q2.2: Classificador BMW-10 (3,4,5 e Outros) com ResNet18

In [1]:
!pip -q install ultralytics opencv-python scipy scikit-learn matplotlib torch torchvision

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

os.makedirs("outputs", exist_ok=True)
print("OK: deps + outputs/")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.9 MB/s eta 0:00:00
OK: deps + outputs/


In [2]:
Q1_IMAGE_PATH  = "/content/q1.jpg"       # imagem dos grãos
Q21_IMAGE_PATH = "/content/q2_1.jpg"      # imagem pra YOLO (pessoas)

## Q1 — OpenCV (Contagem de objetos)

In [3]:
img = cv2.imread(Q1_IMAGE_PATH)
if img is None:
    raise FileNotFoundError(f"Não abriu a imagem: {Q1_IMAGE_PATH}")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)

_, th = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Se ficar invertido, descomente:
# th = 255 - th

kernel = np.ones((3,3), np.uint8)
th_clean = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=1)

cnts, _ = cv2.findContours(th_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

min_area = 80  # ajuste rápido se necessário
valid = [c for c in cnts if cv2.contourArea(c) > min_area]

out = img.copy()
cv2.drawContours(out, valid, -1, (0, 255, 0), 2)
cv2.putText(out, f"Count: {len(valid)}", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

cv2.imwrite("outputs/q1_binary.png", th_clean)
cv2.imwrite("outputs/q1_result.png", out)

print("Q1 Count:", len(valid))
print("Saved: outputs/q1_binary.png, outputs/q1_result.png")

Q1 Count: 87
Saved: outputs/q1_binary.png, outputs/q1_result.png


## Q2.1 — YOLOv8 (Contagem de pessoas)

## Q2.2 — Classificador BMW-10 (3,4,5 e Outros)